## Workflows

1. Update `config/config.yaml`
2. Update `secrets.yaml` [if applicable]
3. Update `params.yaml` [if applicable]
4. Update ENTITY: `src/KDClassification/entity/config-entity.py`
5. Update `ConfigurationManager`: `src/KDClassification/config/configuration.py`
6. Update `components`: `src/KDClassification/components/model_evaluation_mlflow.py`
7. Update `pipeline`: `src/KDClassification/pipeline/stage_04_model_evaluation.py`
8. Update `main.py`
<!-- 9. Update dvc.yaml
10. app.py -->

In [1]:
import os
os.chdir("../")

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/paridhi3/Kidney-Disease-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="paridhi3"
os.environ["MLFLOW_TRACKING_PASSWORD"]="87f847f70b8b5b73083ff140cdf1cb07ef5d2c55"

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

### 4. Update ENTITY: `src/KDClassification/entity/config-entity.py`

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
from KDClassification.constants import *
from KDClassification.utils.common import read_yaml, create_directories, save_json

### 5. Update `ConfigurationManager`: `src/KDClassification/config/configuration.py`

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/KDC-data",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




### 6. Update `components`: `src/KDClassification/components/model_evaluation_mlflow.py`

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


### 7. Update `pipeline`: `src/KDClassification/pipeline/stage_04_model_evaluation.py`

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-25 15:25:33,438: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-25 15:25:33,443: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-25 15:25:33,446: INFO: common: Created directory at: artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 827s 6s/step - loss: 21.6423 - accuracy: 0.3099
[2024-07-25 15:39:25,165: INFO: common: json file saved at: scores.json]


2024/07/25 15:39:28 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\PARIDHI\AppData\Local\Temp\tmpoyhzl8oz\model\data\model\assets
[2024-07-25 15:39:32,304: INFO: builder_impl: Assets written to: C:\Users\PARIDHI\AppData\Local\Temp\tmpoyhzl8oz\model\data\model\assets]


c:\Users\PARIDHI\Desktop\ALL-ML-PROJECTS\Kidney-Disease-Classification\venv\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


MlflowException: API request to endpoint /api/2.0/mlflow/registered-models/create failed with error code 403 != 200. Response body: ''